# Meet the meat

## Abstract

With increasingly dire climate change forecasts, concerned individuals are asking how they can minimize their carbon footprint. Recent research suggests that reducing one's consumption of meat, in particular beef, is one of the highest impact actions an individual can take. To examine this topic, we will explore the popularity and prevalence of meat in recipes. Specifically, we plan to extract the ingredients from a recipe database and calculate the carbon footprint of recipes

Finally, we hope to directly relate this data to the issue of climate change by estimating a rating reflecting the carbon footprint of meat in recipes and the environmental impact of consumers' diets.

### Imports and libraries

In [2]:
# Import libraries
import re
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import os, os.path as osp


In [3]:
DATA_FOLDER='data'
SAMPLE_DATA_FOLDER = DATA_FOLDER + '/recipePages/'#'/sample_400/'

#### Define carbon footprint of meat ingredients
Animal agriculture is one of the leading sources of the carbon-impact of a recipe. We start by assigning a carbon footprint to each meat ingredient and could later on extend it to other animal products. 
The functions below assign a carbon footprint to each meat ingredient of the recipes.

Source of data: [GreenEatz](https://www.greeneatz.com/foods-carbon-footprint.html)

In [4]:
#Load data from xls file
carbon_footprint_list = pd.read_excel('data/carbon_footprint_protein.xls', sheet_name='meat_dairy_eggs', index_col=0)
carbon_footprint_list = carbon_footprint_list[['Food','CO2 Kilos Equivalent']]
carbon_footprint_list['Food']=['lamb','beef','cheese','pork','turkey','chicken','tuna','egg']
carbon_footprint_list

,Food,CO2 Kilos Equivalent
Rank,,
1,lamb,39.2
2,beef,27.0
3,cheese,13.5
4,pork,12.1
5,turkey,10.9
6,chicken,6.9
7,tuna,6.1
8,egg,4.8


In [5]:
#List of meat ingredients: ingredients of interest that we want to look for in our recipes
meat_products = ['steak','lamb', 'beef', 'cheese', 'pork', 'turkey', 'chicken', 'tuna', 'egg']
#units to recognize
units = ['pound','gram','oz','ounce','kg','kilogram','lb', 'egg' ]

## Data extraction and cleaning

Our recipe dataset contains recipes from the [From Cookies to Cooks](http://infolab.stanford.edu/~west1/from-cookies-to-cooks/), combining recipes from 14 high-traffic websites. We start by extracting all the information we want from the HTML files, that is: title, ingredients and meat or animal protein ingredients, tags, ratings in order to explore the recipes in more detail.


#### Recipe webpage scraping
The websites' HTML sources are rich in information. However, the information we wantfrom these pages is rather limited. We extract the information we need from the websites, clean and pre-process the data and save it as a CSV file for easy retrieval in further processing.

In [6]:
####################
#UPDATE:
#I(Alex) changed the website values so that they are compatible when extracting the website name from the title
#See the function "tolkenize_title"
#####################

def find_website(soup):
    """
    Finds if the page is a recipe and which website it comes from
    """
    is_recipe = True
    
    if 'allrecipes' in soup.title.string.casefold():
        website = 'allrecipes'               
              
    elif 'epicurious' in soup.title.string.casefold():
        website = 'epicurious'
    
    elif 'food network' in soup.title.string.casefold():
        website = 'food network'
        
    elif 'food.com' in soup.title.string.casefold():
        website == 'food.com'
    
    elif 'betty crocker' in soup.title.string.casefold() or 'bettycrocker' in soup.title.string.casefold():
        website = 'betty crocker'
               
    elif 'myrecipes' in soup.title.string.casefold():
        website = 'myrecipes'
    
    elif 'taste.com' in soup.title.string.casefold():
        website = 'taste'

    else:
        website = 'not found'
        is_recipe = False
        
    return is_recipe, website

#### Quantity extraction and conversion
The amounts of each ingredients are expressed in many different units (imperial or metric) depending on the websites, and even on the recipes. Once we have extracted the ingredients and amounts, we need to convert all different quantities to one single weight unit (fixed to kilograms) in order to process the carbon footprint of selected ingredients.

In [7]:
def analyse_page(soup, page):
    """
    Input: 
        soup
        page: 'allrecipes', 'epicurious', 'food network', 'food.com', 'betty crocker', 'myrecipes' , others not implemented yet
    
    Output:
        tags = list of tags assigned to the recipe
        ings = list of ingredients
    """
    ings = []
    ing_wrap=None
    tags = []
    rate = None
    serv = None
    
    if page == 'allrecipes':
        # Extract tags
        #tag_wrappers = soup.find_all(itemprop="recipeCategory") 
        #for tag in tag_wrappers:
        #    tags.append(tag['content'])           
        #if not tag_wrappers:
        #    tag_wrappers = soup.find_all('li', class_="leftnav-middle-title")
            #for lis in soup.find_all('li','ul'):# class_='leftnav-middle-title'):
        #    for tag in tag_wrappers:
        #        if tag.h4.contents[0]=="Related Collections":
        #            print('Collections found')
        #            tags.append(tag['content'])
        #            print(tag)
                    
        # Extract ingredients
        try:
            ing_wrap=soup.find_all('li', class_="plaincharacterwrap ingredient")#1st try
            if ing_wrap:
                for ing in ing_wrap:
                    ings.append(ing.getText())
            else:
                ing_wrap=soup.find_all(itemprop="recipeIngredient") #2nd try
                for ing in ing_wrap:
                    ings.append(ing.getText()) 
        except:
            pass
            #print('Ingredients could not be extracted from '+ page +" recipe")

        # Extract Ratings
        try:
            rate_wrapper=soup.find('p',class_='reviewP')
            rate_wrapper=rate_wrapper.find('img', id="ctl00_CenterColumnPlaceHolder_recipe_ratingStuff_imgRating")
            rate=re.findall(r"[+]?\d*\.\d+|\d+", rate_wrapper['title'])[0]
        except:
            pass
            #print('Ratings could not be extracted from '+ page +" recipe")

        # Extract Number of servings
        try:
            serv=soup.find('span', class_="yield yieldform").text
        except:
            pass
            #print('Serving size could not be extracted from '+ page +" recipe")

        
    elif page == 'epicurious':       
        # Extract tags

        #tag_wrappers = soup.find_all(itemprop="recipeCuisine")
        #for tag in tag_wrappers:
        #    tags.append(tag.getText())    
        #tag_wrappers = soup.find_all(itemprop="recipeCategory")
        #for tag in tag_wrappers:
        #    tags.append(tag.getText())        
        
        # Extract ingredients
        try:            
            ing_wrap=soup.find('div', id="ingredients")
            if ing_wrap:
                for ing in ing_wrap:
                    ings.append(ing.string)
            if None in ings:
                ings=[]
                ing_wrap=soup.find_all('li', class_="ingredient")
                for ing in ing_wrap:
                    ings.append(ing.string)
        except:
            pass
            #print('Ingredients could not be extracted from '+ page +" recipe")
        #extract rating
        try:
            rate_wrapper=soup.find('p',id='prepare_again_rating') 
            rate=re.findall(r"[+]?\d*\.\d+|\d+", rate_wrapper.getText())[0]
            rate=float(rate)*5/100
        except:
            pass
            #print('Rating could not be extracted from '+ page +" recipe")

            #extract serving size 
        try:
            serv=soup.find('span',class_='yield').text
        except:
            pass
            #print('Serving size could not be extracted from '+ page +" recipe")

    elif page == 'food network':  
        # Extract tags
        #tag_wrappers = soup.find_all(class_="btn grey-tags")
        #if not tag_wrappers:
        #    tag_wrappers = soup.find_all(class_='tag')
        #for tag in tag_wrappers:   ##THIS ONE?
        #    tags.append(tag.getText())
            
        #for t in soup.find_all(id = "recipe-filedin"):  #OR THIS ONE?
        #    for li in t.find_all('li'):
        #        for a in li.find_all('a'):
        #            tags.append(a.getText())
        
        #for tag in tag_wrappers:
        #    tags.append(tag.getText())      
        
        # Extract ingredients
        try:
            ing_wrap=soup.find_all('li',class_='ingredient')
            for ing in ing_wrap:
                ings.append(ing.text)
        except:
            pass
            #print('Ingredients could not be extracted from '+ page +" recipe")

        #Extract rating
        try:
            rate_wrapper=soup.find('div', class_='rm-block lead hreview-aggregate review')
            rate=rate_wrapper.find('div')['title']
        except:
            pass
            #print('Rating could not be extracted from '+ page +" recipe")
            
        #extract serving size
        try:
            serv0=soup.find('div', id='recipe-meta')
            serv1=serv0.find_all('dd', class_="clrfix")
            for ser in serv1:
                if any(char.isdigit() for char in ser.get_text()):
                #'serving' in ser.get_text() or 'cup' in ser.get_text():  ##NEED A BETTER WAY TO DO THIS (00a1cb2c972a31e50718971edd070a50)
                    serv=ser.get_text()
        except:
            pass
            #print('Serving could not be extracted from '+ page +" recipe")

    elif page == 'food.com':      
        # Extract tags
            #not found          
        # Extract ingredients
        try:
            ing_wrap=soup.find_all('li', class_="ingredient")
            if ing_wrap:
                for ing in ing_wrap:
                    ings.append((ing.find('span',class_='value').text+ ' '+ing.find('span',class_='type').text + ' ' + ing.find('span', class_='name').text))
            else:
                ing_wrap=soup.find_all(class_="name")
                for ing in ing_wrap:
                    ings.append(ing.getText())
        except:
            pass
            #print('Ingredients could not be extracted from '+ page +" recipe")


    elif page == 'betty crocker':   
        # Extract tags
            #not found    
            
        # Extract ingredients
        try:
            ing_wrap=soup.find_all('dl', class_='ingredient')
            for ing in ing_wrap:
                ings.append(ing.getText())
        except:
            pass
            #print('Ingredients could not be extracted from '+ page +" recipe")

        #Extract serving size
        try:
            serv=(soup.find('meta', itemprop='recipeYield'))['content'] #tag attribute 'content'
        except:
            pass
            #print('Serving size could not be extracted from '+ page +" recipe")

    elif page == 'myrecipes':
        # Extract tags
        #tag_wrappers = soup.find_all(itemprop="recipeType")
        #for tag in tag_wrappers:
        #    tags.append(tag.getText())  
            
        # Extract ingredients
        try:
            ing_wrap=soup.find_all(itemprop="ingredient")
            for ing in ing_wrap:
                ings.append(ing.text)
        except:
            pass
            #print('Ingredients could not be extracted from '+ page +" recipe")

        #extract serving size
        try:
            serv=soup.find('span', itemprop='yield').text
        except:
            pass
            #print('Serving size could not be extracted from '+ page +" recipe")

    elif page == 'taste':
        #Extract ingredients
        try:
            ing_wrap = soup.find('div', class_="module recipe-ingredients")
            ing_wrap=ing_wrap.find_all('li')
            #print(ing_wrap)
            for ing in ing_wrap:
                ings.append(ing.text)
        except:
            pass
            #print('Ingredients could not be extracted from '+ page +" recipe")
        
        try:
            rate_wrapper=soup.find('span', itemprop='rating')
            rate=rate_wrapper.getText()
        except:
            pass
            #print('Rating could not be extracted from '+ page +" recipe")

        
        #extract serving size
        try:
            serv=soup.find('h2', class_="ingredients").text
        except: 
            pass
            #print('Serving size could not be extracted from '+ page +" recipe")

            
    #other websites    
        # Extract tags   
        # Extract ingredients 
        
    #if not ing_wrap:  #return warning if website is recognized but format/data extraction is not successful
    #    print('Not a recipe format')  
    #    print('*******')
    
    #if not tags:
        #print('no tags found :( ')
        
    return tags, ings, rate, serv


In [8]:
def check_quantity(quant_str):
    """
    Cleans input string and extracts numerical values
    Outputs cleaned string, array of numerical values and sum of numerical values
    """
    quant_str=quant_str.replace("½",".5")
    quant_str=quant_str.replace("1/2",".5")
    quant_str=quant_str.replace("1/3", '.33')
    quant_str=quant_str.replace('1/4','.25')
    quant_str=quant_str.replace('3/4','.75')
    

    if 'dozen' in quant_str: #likely going to be in the form "2 dozen"
        
        quant_str=quant_str.replace('dozen', 'x12')
        
        #matches positive decimals or whole numbers
        quant_vals=re.findall(r"[+]?\d*\.\d+|\d+", quant_str)
        total_quant=np.prod([float(i) for i in quant_vals])
    
    else: 
        
        #matches positive decimals or whole numbers
        quant_vals=re.findall(r"[+]?\d*\.\d+|\d+", quant_str)
        total_quant=np.sum([float(i) for i in quant_vals])    

    return quant_str, quant_vals, total_quant


def convert_to_kg(quant, unit):
    """
    Converts any input unit (kg, lb, grams, ounces) to kilograms
    """
    
    if (unit=='kilogram') or (unit=='kg'):
        amnt_kg=quant
        #print(quant,'kg')
    elif (unit=='pound') or (unit=='lb') or (unit=='lbs') or (unit=='pounds'):
        amnt_kg=quant/2.205
        #print(amnt_kg,'kg')
    elif(unit=='g') or (unit=='gram') or (unit =='grams'):
        amnt_kg=quant/1000
        #print(amnt_kg,'kg')     
    elif(unit=='oz') or (unit=='ounce'):
        amnt_kg=quant/35.274
        #print(amnt_kg, 'kg')
    elif(unit=='egg'):
        amnt_kg=quant*0.006 #1 egg weighs aproximately 60g
    else:
        pass
        #print('unit not recognized')
        
    return(amnt_kg)

def contains_meat_ingredients(ings_in, meat_products_in):
    contains_meat=False
    meat_ingredients=[]
    meat_ingredients_str=[]
    for i in ings_in:
        for meat_product in meat_products_in:
            if i != None:
                if meat_product in i.casefold(): 
                    contains_meat=True
                    meat_ingredients.append(meat_product) 
                    meat_ingredients_str.append(i)
                    
    return contains_meat, meat_ingredients, meat_ingredients_str

def extract_meat(meat_ings_str):
    """
    Inputs: 
    ings_in= list of ingredients (and quantities)
     
    Outputs:
    ing_amnt_out = list of corresponding quanities of meat ingredients in kg (=0 if unit not recognized)
    
    """

    ing_amnt_out=[]
    #extract amount from string and convert to kg
    for meat_i in meat_ings_str:
        meat_i_quant_kg=0
        meat_i, quantity_vals, total_quantity=check_quantity(meat_i) #pass string, return cleaned string and total quantity
        #find appropriate units and convert to kg
        for u in units:
            if u in meat_i.casefold():
                meat_i_quant_kg = convert_to_kg(total_quantity,u)
        ing_amnt_out.append(meat_i_quant_kg)
        #if meat_i_quant_kg==0:
        #    print('Units not recognized for: '+meat_i)
                    
    return ing_amnt_out

def normalize_servings(ing_amount, servs):
    #input is ingredient quantity in kg
    #interpret servings as a total quantity using the check_quantity function
    
    ing_norm=[]
    str_servs, val_servs, tot_servs = check_quantity(servs)
    for ing in ing_amount:
        ing_norm.append(ing/tot_servs)
    #print('normalizing by number of servings: ', tot_servs)
    return ing_norm    




In [9]:
# calculate carbon footprint
def compute_carbon_fp (meat_ingred, meat_quant, meat_carbon_list):
    """
    
    #SHOULD WE USE A CONSTANT WEIGHT IF IT CONTAINS THE INGREDIENT AND THEN A WEIGHT DEPENDING ON HOW MUCH IT CONTAINS?
    #ALTERNATIVELY WE COULD ESTIMATE HOW MUCH OF AN INGEDIENT IS BASED ON THE MEDIAN VALUE
    #OR DROP ALL RECIPES WHERE WE DON'T KNOW THE QUANTITY
    
    
    input: 
    meat_ingred= types of meat in recipe
    meat_quant = quantity (normalized by serving size) of meat ingredient in kg
    meat_carbon_list = carbon footprint/kg of ingredients of interest
    output:
    df_merge = dataframe with the ingredient type and its corresponding carbon footprint
    carbon_footprint (sum of all ingredients' carbon footprint)
    
    """
    for i in np.arange(len(meat_ingred)):
        if 'steak' in meat_ingred[i].casefold():
            meat_ingred[i]='beef'
            
    #create data frame and merge with carbon footprint
    df=pd.DataFrame({'meat item':meat_ingred, 'item quantity':meat_quant})        
    df=df.groupby(df['meat item']).sum()
    df_merge=df.merge(meat_carbon_list, how='left', left_on='meat item', right_on='Food')
    df_merge['carbon footprint']=df_merge['item quantity']*df_merge['CO2 Kilos Equivalent']
    #print(df_merge)
    return df_merge[['Food','carbon footprint']], df_merge['carbon footprint'].sum()
    

compute_carbon_fp(['beef','egg','cheese', 'steak'], [1.5,.02,.3,.39], carbon_footprint_list)

(     Food  carbon footprint
 0    beef            51.030
 1  cheese             4.050
 2     egg             0.096, 55.175999999999995)

In [18]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    


### Data extraction and cleaning loop
Below we extract the data from the recipes of our html dataset and save it in dataframes. Our goal here is to extract the ingredients and assign a carbon-impact rating to the highest impact ingredients (meat or animal protein) in the recipes.

To extract protein-rich ingredients from animal source in order to calculate the main carbon footprint of the recipe, we use an extra database listing the main protein sources and carbon impact. Source of data: [GreenEatz](https://www.greeneatz.com/foods-carbon-footprint.html)

In [ ]:
#Loop for all recipes in folder
# data has following row structure
# RecipeName as Identifier - bool contains_meat - list of co2 ingredients - carbonFootprint - ingredients
data=[]
step=0
total_num=93701

verbose = 0 #verbose outputs
printProgressBar(0, total_num, prefix = 'Progress:', suffix = 'Complete', length = 50)

for filename in os.listdir(SAMPLE_DATA_FOLDER):
    with open(SAMPLE_DATA_FOLDER+filename) as f:
        isTrue=False
        count_exceptions=0

        try:
            page = f.read()
            soup = BeautifulSoup(page, 'html.parser')
            
            #check webpage and extract ingredients if recognised as recipe
            is_recipe, website = find_website(soup)
            #print('Recipe Analysed: '+soup.title.string)
            #print('filename: '+filename)

            if is_recipe:

                #tags, ingredients = analyse_page(soup, website)
                tags, ingredients, rating, servings= analyse_page(soup, website)
                if ingredients:
                    
                    
                    has_meat, meat_ingredients, meat_ingredients_str = contains_meat_ingredients(ingredients, meat_products)

                    if has_meat:

                        #Extract meat ingredients and quantities in kg
                        ingredient_quant_kg = extract_meat(meat_ingredients_str)
                        ingredient_quant_norm = normalize_servings(ingredient_quant_kg, servings)
                        #calculate carbon footprint
                        fp_per_ingredient, fp_total = compute_carbon_fp(meat_ingredients, ingredient_quant_norm, carbon_footprint_list)
                    else:
                        ingredient_quant_kg = 0
                        ingredient_quant_norm = 0
                        fp_per_ingredient = 0
                        fp_total = 0

                    if verbose: 
                        #print('Recipe Analysed: '+soup.title.string)
                        print('{0} Ingredients: '.format(len(ingredients)))
                        print('contains meat:'+str(has_meat))
                        if has_meat:
                            print(meat_ingredients_str)
                            print('ingredient_quantity (kg)= ',ingredient_quant_kg)
                            print('ingredient quantity (kg) per serving = ', ingredient_quant_norm )
                            print('carbon footprint of recipe = ', fp_total)

                        print('{0} tags:'.format(len(tags)))
                        print(tags)
                        print('number of servings= ', servings)
                        print('rating = ', rating)
                            
                    data.append([soup.title.string, has_meat, meat_ingredients, ingredient_quant_norm, fp_total, rating, website])

            else:
                pass
                #print('not a recipe')
        except:
            count_exceptions=count_exceptions+1
    
    
    printProgressBar(step, total_num, prefix = 'Progress:', suffix = 'Complete', length = 50)    
    
    #REMOVE IF YOU WANT MORE SAMPLES
    if step%1000==0: #every 100 files, save progress
        column_labels=['Recipe Title', 'Has meat', 'Meat types', 'Meat quantity (kg)', 'Carbon footprint','Rating','Website']#'Tags']#missing: 'Carbon footprint', 'Rating', 'Tags'
        recipes_df = pd.DataFrame(data, columns = column_labels)
        recipes_df.to_csv(DATA_FOLDER+'/recipes_data.csv',  index=False)
        #print('saving files, iteration =',step)

    step=step+1 #next step
    if step>total_num: #next step is beyond dataset
        column_labels=['Recipe Title', 'Has meat', 'Meat types', 'Meat quantity (kg)', 'Carbon footprint','Rating','Website']#'Tags']#missing: 'Carbon footprint', 'Rating', 'Tags'
        recipes_df = pd.DataFrame(data, columns = column_labels)
        recipes_df.to_csv(DATA_FOLDER+'/recipes_data.csv',  index=False) #save all data
        break
    

#column_labels=['Recipe Title', 'Has meat', 'Meat types', 'Meat quantity (kg)', 'Carbon footprint','Rating','Website']#'Tags']#missing: 'Carbon footprint', 'Rating', 'Tags'
#recipes_df = pd.DataFrame(data, columns = column_labels)


#recipes_df

C:\Users\alexk\Anaconda\envs\ada\lib\site-packages\ipykernel\__main__.py:101: RuntimeWarning: divide by zero encountered in double_scalars


C:\Users\alexk\Anaconda\envs\ada\lib\site-packages\ipykernel\__main__.py:101: RuntimeWarning: invalid value encountered in double_scalars


In [17]:
#save the data as csv for in depth analysis
#recipes_df.to_csv(DATA_FOLDER+'/recipes_data.csv',  index=False)
step

1001

***STOP****
All code below this line is experimental and should be deleted in the final version.

In [ ]:
#code to test individual files: DELETE IN FINAL VERSION
with open(SAMPLE_DATA_FOLDER+'00bb0f80a1286ab29611a34540a8f54e.html') as f:
    page = f.read()
    soup = BeautifulSoup(page, 'html.parser')
    rate_wrapper=soup.find('span', itemprop='rating')
    rate=rate_wrapper.getText()
    print(rate_wrapper)
    print(rate)
    

In [ ]:
#Betty crocker ratings are a pain
#so are myrecipes

In [ ]:

#CODE FOR TESTING
    
            #check webpage and extract ingredients if recognised as recipe
    is_recipe, website = find_website(soup)
    print('Recipe Analysed: '+soup.title.string)
    print('WEBSITE= ', website)
    #print(soup.prettify())
    tolken_title=tolkenize_title(soup.title.string, website)

    #tags, ingredients = analyse_page(soup, website)
    tags, ingredients, rating, servings= analyse_page(soup, website)
    has_meat, meat_ingredients, meat_ingredients_str = contains_meat_ingredients(ingredients, meat_products)
    #Extract meat ingredients and quantities in kg
    ingredient_quant_kg = extract_meat(meat_ingredients_str)
    print(servings)
    print('contains meat:'+str(has_meat))
    print(meat_ingredients_str)                  
    print('{0} Ingredients: '.format(len(ingredients)))
    print(ingredients)
    print('{0} tags:'.format(len(tags)))
    print(tags)
    print('does this recipe contain meat? ', has_meat)
    print('meat ingredients=', meat_ingredients_str)
    print('ingredient_quantity (kg)= ',ingredient_quant_kg)
if has_meat:
     #Extract meat ingredients and quantities in kg
    ingredient_quant_kg = extract_meat(meat_ingredients_str)
    ingredient_quant_norm = normalize_servings(ingredient_quant_kg, servings)
                        #calculate carbon footprint
    fp_per_ingredient, fp_total = compute_carbon_fp(meat_ingredients, ingredient_quant_norm, carbon_footprint_list)
else:
    ingredient_quant_kg = 0
    ingredient_quant_norm = 0
if verbose: 
    #print('Recipe Analysed: '+soup.title.string)                        
    print('tolkenized title : ', tolken_title)

                        
    print('contains meat:'+str(has_meat))
    print(meat_ingredients_str)
                        
    print('{0} Ingredients: '.format(len(ingredients)))
                        #print(ingredients)

    print('{0} tags:'.format(len(tags)))
    print(tags)
    print('number of servings= ', servings)
    print('does this recipe contain meat? ', has_meat)
                        #print('ingredients = ',ingredients)
    print('meat ingredients=', meat_ingredients_str)
    print('ingredient_quantity (kg)= ',ingredient_quant_kg)
    print('ingredient quantity (kg) per serving = ', ingredient_quant_norm )
    print('carbon footprint of recipe = ', fp_total)
                            